<div align="center">

# **Prova de Estágio - Python e SQL**
### *Otávio Feliciano Assumpção*  

##### *08 de Outubro de 2024*

</div>

---

Este documento contém as respostas às perguntas da prova, que tem como objetivo avaliar meus conhecimentos em SQL e Python, utilizando situações práticas e complexas. Esse material está dividido em duas partes:

- Na **primeira parte**, responderei às questões relacionadas a **SQL**;
- Na **segunda parte**, abordarei as questões focadas em **Python**.

Instruções técnicas sobre as versões dos pacotes utilizados e clonagem do repositório estão disponíveis no arquivo **README.md**, disponível no [repositório do GitHub](https://github.com/otavioassumpcao/Case__SQL_Python).

---


<div align="center">

## **Parte 01: SQL**

</div>

---

Antes de começarmos de fato, faz-se necessário configurar o ambiente e preparar as bases de dados para que possamos executar as consultas em SQL diretamente no Jupyter Notebook. Nesta seção, vamos:

1. Carregar as extensões necessárias para executar SQL no Jupyter;
2. Preparar um banco de dados SQLite local e carregar os dados dos arquivos CSV nele;
3. Estabelecer a conexão entre o Jupyter Notebook e o banco de dados SQLite para executar as consultas SQL.

**1. Carregar a extensão SQL no Jupyter**

In [ ]:
%load_ext sql

Aqui, estamos carregando a extensão `sql` no Jupyter Notebook. Essa extensão é parte do pacote `ipython-sql` (também chamado de `jupysql`).

**2. Importar bibliotecas e preparar os dados**

In [45]:
# Importando pacotes
import pandas as pd
import sqlite3

# Carregando os arquivos CSV
clientes = pd.read_csv('Bases/clientes.csv')
vendas = pd.read_csv('Bases/vendas.csv')

# Conectando ao banco de dados SQLite e criando o arquivo
conn = sqlite3.connect('clientes_vendas.db') # Aqui o arquivo clientes_vendas.db é criado
clientes.to_sql('clientes', conn, if_exists='replace', index=False)
vendas.to_sql('vendas', conn, if_exists='replace', index=False)
conn.close()


**Importando as bibliotecas**:
- `pandas`: Para carregar e manipular os dados dos arquivos CSV.
- `sqlite3`: Para interagir com um banco de dados SQLite.

**Carregando os arquivos CSV**:
- Os arquivos `clientes.csv` e `vendas.csv` são carregados como DataFrames utilizando o pandas.

**Criando o banco de dados SQLite**:
- Uma conexão é criada com o banco de dados SQLite utilizando ``sqlite3.connect()``. O banco de dados será salvo no arquivo `clientes_vendas.db`.
- Utiliza o método `to_sql()` para transferir os dados dos DataFrames para o banco de dados, criando as tabelas clientes e vendas.
- Fecha a conexão com o banco de dados para garantir que todas as operações foram finalizadas corretamente.

**3. Conectar o Jupyter ao banco de dados SQLite**

In [46]:
%sql sqlite:///clientes_vendas.db


Estabelecemos a conexão entre o ambiente do Jupyter Notebook e o banco de dados SQLite que foi criado anteriormente. Ao usar o comando `%sql`, informamos ao Jupyter que queremos executar comandos SQL e fazer a conexão com o banco de dados `clientes_vendas.db`.

Agora sim, podemos partir para a resolução!

---

#### **Dando uma olhada nos DataFrames**


In [47]:
%%sql

-- Query para listar todos os clientes
SELECT * FROM clientes;

id_cliente,nome_cliente,cidade
1,João Souza,Rio de Janeiro
2,Ana Costa,São Paulo
3,Carlos Lima,Belo Horizonte


In [48]:
%%sql

-- Query para listar todas as vendas
SELECT * FROM vendas;

id_venda,id_cliente,data_venda,valor_total
1,1,2023-05-10,200
2,2,2023-06-12,450
3,1,2023-07-22,130
4,3,2023-08-15,500
5,2,2023-09-05,670


---

#### **Questão 01. `SELECT` básico com filtro:**  
*Retorne o nome dos clientes e o valor total de suas vendas realizadas em 2023, utilizando `JOIN` e `WHERE`.*


In [49]:
%%sql

-- Seleciona o nome dos clientes e o valor total das vendas de 2023
SELECT clientes.nome_cliente, SUM(vendas.valor_total) AS total_vendas

-- As tabelas 'clientes' e 'vendas' são combinadas usando JOIN
FROM clientes
JOIN vendas ON clientes.id_cliente = vendas.id_cliente

-- Filtra as vendas realizadas apenas entre 1º de janeiro e 31 de dezembro de 2023
WHERE vendas.data_venda BETWEEN '2023-01-01' AND '2023-12-31'

-- Agrupa os resultados por cliente, somando as vendas de cada cliente
GROUP BY clientes.nome_cliente;


nome_cliente,total_vendas
Ana Costa,1120
Carlos Lima,500
João Souza,330


#### **Questão 02. `JOIN` + `GROUP BY`:**  
*Retorne o total de vendas por cidade, agrupando os dados e ordenando o resultado pelo total de vendas em ordem decrescente.*

In [50]:
%%sql

-- Seleciona a cidade dos clientes e o valor total das vendas em cada cidade
SELECT clientes.cidade, SUM(vendas.valor_total) AS total_vendas

-- Faz o JOIN entre as tabelas clientes e vendas, com base no id_cliente
FROM clientes
JOIN vendas ON clientes.id_cliente = vendas.id_cliente

-- Agrupa os resultados por cidade, somando as vendas
GROUP BY clientes.cidade

-- Ordena os resultados pelo valor total das vendas em ordem decrescente
ORDER BY total_vendas DESC;

cidade,total_vendas
São Paulo,1120
Belo Horizonte,500
Rio de Janeiro,330


#### **Questão 03.  `INSERT` e `UPDATE`:**
*Insira um novo cliente e uma nova venda associada a ele, e depois faça um `UPDATE` para alterar o valor de uma venda existente*

1. **Inserindo um novo cliente**\
    *(sem risco de duplicação)*

In [51]:
%%sql

-- Insere o cliente somente se ele não estiver registrado, caso contrário, não faz nada 
-- (não corre o riso de duplicar um cliente)

INSERT INTO clientes (id_cliente, nome_cliente, cidade)
SELECT 4, 'Otávio Assumpção', 'Campo Grande'
WHERE NOT EXISTS (
    SELECT 1 FROM clientes WHERE id_cliente = 4
);

-- Visualizando 
SELECT * FROM clientes;

id_cliente,nome_cliente,cidade
1,João Souza,Rio de Janeiro
2,Ana Costa,São Paulo
3,Carlos Lima,Belo Horizonte
4,Otávio Assumpção,Campo Grande


2. **Inserindo uma data de compra para o novo cliente**\
    *(sem risco de duplicação de registro da compra)*

In [52]:
%%sql

-- Insere uma nova venda para o cliente 4 apenas se o id_venda não existir
-- Dessa forma, não corremos o risco de registrar a mesma venda mais de uma vez
INSERT INTO vendas (id_venda, id_cliente, valor_total, data_venda)
SELECT 6, 4, 280.00, '2024-01-10'
WHERE NOT EXISTS (
    SELECT 1 FROM vendas WHERE id_venda = 6
);

-- Visualizando
SELECT * FROM vendas;

id_venda,id_cliente,data_venda,valor_total
1,1,2023-05-10,200
2,2,2023-06-12,450
3,1,2023-07-22,130
4,3,2023-08-15,500
5,2,2023-09-05,670
6,4,2024-01-10,280


3. **Alterando o valor de uma venda existente**\
    *(A venda 6 vai de 280 para 380)*

In [37]:
%%sql

-- Atualiza o valor total da venda com id_venda 6 para 380
UPDATE vendas
SET valor_total = 380
WHERE id_venda = 6;

-- Visualizando
SELECT * FROM vendas;

id_venda,id_cliente,data_venda,valor_total
1,1,2023-05-10,200
2,2,2023-06-12,450
3,1,2023-07-22,130
4,3,2023-08-15,500
5,2,2023-09-05,670
6,4,2024-01-10,380


#### **Questão 04.   `DISTINCT` e `GROUP BY`:**
*Retorne a lista de cidades distintas com clientes que fizeram mais de uma venda*

In [53]:
%%sql

-- Retorna as cidades distintas com clientes que fizeram mais de uma venda
SELECT DISTINCT clientes.cidade
FROM clientes
JOIN vendas ON clientes.id_cliente = vendas.id_cliente
GROUP BY clientes.id_cliente, clientes.cidade
HAVING COUNT(vendas.id_venda) > 1;

cidade
Rio de Janeiro
São Paulo


---

<div align="center">

## **Parte 02: Python com Pandas e Conceitos**

</div>

---

#### **Questão 01. DataFrames e Pandas :**
*A partir de um dicionário de dados brutos de vendas, crie um DataFrame e exiba as 5 primeiras linhas.*

In [69]:
import pandas as pd # Já háviamos importando anteriormente, apenas para reforçar

df_vendas = pd.read_csv('Bases/vendas.csv') # Função read.csv() já retona como DataFrame
df_clientes = pd.read_csv('Bases/clientes.csv') # Vamos utilizar essa base nos próximos itens

# Exibir as 5 primeiras linhas do DataFrame de vendas
df_vendas.head() # .head() retorna os 5 primeiros por default


,id_venda,id_cliente,data_venda,valor_total
0,1,1,2023-05-10,200
1,2,2,2023-06-12,450
2,3,1,2023-07-22,130
3,4,3,2023-08-15,500
4,5,2,2023-09-05,670


#### **Questão 02. Filtros em DataFrames:**
*Filtre o DataFrame para mostrar apenas vendas acima de 300 reais.*

In [70]:
# Sem criar um novo df e sem editar o df original
df_vendas[df_vendas['valor_total'] > 300]

,id_venda,id_cliente,data_venda,valor_total
1,2,2,2023-06-12,450
3,4,3,2023-08-15,500
4,5,2,2023-09-05,670


In [71]:
# Como podemos ver, o df original não foi alterado
df_vendas

,id_venda,id_cliente,data_venda,valor_total
0,1,1,2023-05-10,200
1,2,2,2023-06-12,450
2,3,1,2023-07-22,130
3,4,3,2023-08-15,500
4,5,2,2023-09-05,670


#### **Questão 03. groupby em Pandas:**
*Agrupe os dados pelo id_cliente e calcule o total de vendas por cliente.*

In [72]:
# Dando uma olhada no df dos clientes (o mesmo que utilizamos nas questões de SQL)

df_clientes

,id_cliente,nome_cliente,cidade
0,1,João Souza,Rio de Janeiro
1,2,Ana Costa,São Paulo
2,3,Carlos Lima,Belo Horizonte


Aqui optei por criar novos DataFrames ao invés de simplesmente filtrar para ser possível adicionar novas informações, o que será útil para o próximo ítem.

In [73]:
# Criando um df novo (df_master) que compila todas as informações num lugar só (registra todas as operações de venda com todas as variáveis associadas a cada uma delas, vamos utilizar posteriormente para salvar os novos inputs).
df_master = pd.merge(df_vendas, df_clientes, on='id_cliente')

# Criando outro df (df_total_clientes) que mostra o total em compras de cada cliente.
df_total_cliente = df_master.groupby(['id_cliente', 'nome_cliente'])['valor_total'].sum().reset_index()
df_total_cliente

,id_cliente,nome_cliente,valor_total
0,1,João Souza,330
1,2,Ana Costa,1120
2,3,Carlos Lima,500


#### **Questão 04. Classes e Métodos:**
*Crie uma classe Cliente, com métodos para adicionar e exibir o saldo de um cliente*

In [74]:
# Criando a classe Cliente
class Cliente:
    def __init__(self, id_cliente=None, nome_cliente=None, cidade=None):
        self.id_cliente = id_cliente
        self.nome_cliente = nome_cliente
        self.cidade = cidade

    def registrar_venda(self, df_master):
        # Pedindo o ID do cliente via input
        self.id_cliente = int(input("Digite o ID do cliente: "))

        # Verifica se o cliente já existe no DataFrame
        if self.id_cliente in df_master['id_cliente'].values:
            print("Cliente encontrado! Insira os detalhes da nova venda.")
            # Para clientes existentes, pede apenas valor da compra e data
            valor_compra = float(input("Digite o valor da nova compra: "))
            data_venda = input("Digite a data da venda (formato YYYY-MM-DD): ")
            self.nome_cliente = df_master.loc[df_master['id_cliente'] == self.id_cliente, 'nome_cliente'].values[0]
            self.cidade = df_master.loc[df_master['id_cliente'] == self.id_cliente, 'cidade'].values[0]
        else:
            print("Novo cliente! Por favor, insira as informações do cliente.")
            # Para novos clientes, pede nome, cidade, valor da compra e data
            self.nome_cliente = input("Digite o nome do cliente: ")
            self.cidade = input("Digite a cidade do cliente: ")
            valor_compra = float(input("Digite o valor da primeira compra: "))
            data_venda = input("Digite a data da venda (formato YYYY-MM-DD): ")

        # Adiciona a venda ao df_master (definido a seguir)
        df_master = self.adicionar_venda(df_master, valor_compra, data_venda)

        return df_master

    def adicionar_venda(self, df_master, valor_compra, data_venda):
        # Gera o novo id_venda com base no último id_venda do DataFrame
        if len(df_master) > 0:
            novo_id_venda = df_master['id_venda'].max() + 1
        else:
            novo_id_venda = 1

        # Verifica se o cliente já existe no df_master
        if self.id_cliente in df_master['id_cliente'].values:
            # Cliente existente: Adiciona apenas a nova compra
            nova_venda = pd.DataFrame({
                'id_venda': [novo_id_venda],
                'id_cliente': [self.id_cliente],
                'data_venda': [data_venda],
                'valor_total': [valor_compra],
                'nome_cliente': [self.nome_cliente],  # Nome já existente
                'cidade': [self.cidade]  # Cidade já existente
            })
        else:
            # Novo cliente: Adiciona todos os dados relevantes
            nova_venda = pd.DataFrame({
                'id_venda': [novo_id_venda],
                'id_cliente': [self.id_cliente],
                'data_venda': [data_venda],
                'valor_total': [valor_compra],
                'nome_cliente': [self.nome_cliente],
                'cidade': [self.cidade]
            })

        # Concatena a nova venda ao df_master
        df_master = pd.concat([df_master, nova_venda], ignore_index=True)
        return df_master

    def exibir_saldo(self, df_total_cliente):
        # Busca o saldo atualizado do cliente e o nome do cliente a partir do DataFrame
        if self.id_cliente in df_total_cliente['id_cliente'].values:
            saldo = df_total_cliente.loc[df_total_cliente['id_cliente'] == self.id_cliente, 'valor_total'].values[0]
            nome_cliente = df_total_cliente.loc[df_total_cliente['id_cliente'] == self.id_cliente, 'nome_cliente'].values[0]
            print(f"Cliente {nome_cliente} (ID: {self.id_cliente}) tem um saldo de: R$ {saldo}")
        else:
            print(f"Cliente (ID: {self.id_cliente}) não encontrado.")

O que está sendo feito aqui é basicamente o seguinte: quando o método `registrar_venda` é chamado, a classe pede o ID do cliente e verifica se ele já está no sistema. Se ja estiver, só é solicitado o valor da compra e a data que foi feita, já que as outras infos são fixas para cada cliente. Se o cliente for novo, a classe pede todas as informações - nome, cidade, valor da compra e data.

Depois disso, o método `adicionar_venda` é chamado para inserir a venda no df_master, que criamos no exercício anterior, criando um novo ID de venda e registrando tudo no sistema.

Por último, o método `exibir_saldo` permite verificar o saldo total de um cliente, mostrando o nome e o valor total de suas compras. Ele informa que o cliente não foi encontrado se o ID inputado não estiver na base master.

Resumindo, a classe executa todo o processo de registro de vendas e exibição do saldo dos clientes, verificando se o cliente já existe ou não e organizando as informações no sistema.

A seguir, vamos realizar alguns testes. Primeiro, vamos registar a compra de um cliente que já está na base, o cliente de com id sendo 1 (João Souza), vamos registar uma compra de R$500,00 no nome dele no dia 28/12/2020

In [75]:
# Criando uma instância de Cliente e registrando a venda
cliente = Cliente()
df_master = cliente.registrar_venda(df_master)

# Atualizando df_total_cliente
df_total_cliente = df_master.groupby(['id_cliente', 'nome_cliente'])['valor_total'].sum().reset_index()

# Exibindo o saldo do cliente
cliente.exibir_saldo(df_total_cliente)

Cliente encontrado! Insira os detalhes da nova venda.
Cliente João Souza (ID: 1) tem um saldo de: R$ 830.0


Se repararmos, o novo saldo do João Souza agora é R$830,00 (voltando um pouco, no exercício 3 é possível ver que o saldo era de R$330, o que nos mostra que a operação de atualização da base funcionou).

Vamos agora adicionar um novo cliente, Otávio Assumpção, de ID 4 e que mora em Campo Grande. Nele, vamos registrar uma venda de R$280,00 no dia 01/01/2023.

In [76]:
# Criando uma instância de Cliente e registrando a venda
cliente = Cliente()
df_master = cliente.registrar_venda(df_master)

# Atualizando df_total_cliente
df_total_cliente = df_master.groupby(['id_cliente', 'nome_cliente'])['valor_total'].sum().reset_index()

# Exibindo o saldo do cliente
cliente.exibir_saldo(df_total_cliente)

Novo cliente! Por favor, insira as informações do cliente.
Cliente Otávio Assumpção (ID: 4) tem um saldo de: R$ 280.0


Aqui, podemos reparar que o saldo do novo cliente é justamente o valor da compra que registramos, pois como se trata de um novo cliente, não havia nenhum registro anteriormente.

Vamos agora ver como está nossa base de compras totais por cliente:

In [77]:
df_total_cliente

,id_cliente,nome_cliente,valor_total
0,1,João Souza,830.0
1,2,Ana Costa,1120.0
2,3,Carlos Lima,500.0
3,4,Otávio Assumpção,280.0


É possível perceber que o saldo do João Souza aumentou em R$500,00, e que agora temos o Otávio Assumpção na nossa base, relfetindo as operações realizadas anteriormente.

Por último, vamos analisar nossa `df_master`, que registra todas as transações:

In [78]:
df_master

,id_venda,id_cliente,data_venda,valor_total,nome_cliente,cidade
0,1,1,2023-05-10,200.0,João Souza,Rio de Janeiro
1,2,2,2023-06-12,450.0,Ana Costa,São Paulo
2,3,1,2023-07-22,130.0,João Souza,Rio de Janeiro
3,4,3,2023-08-15,500.0,Carlos Lima,Belo Horizonte
4,5,2,2023-09-05,670.0,Ana Costa,São Paulo
5,6,1,2020-12-28,500.0,João Souza,Rio de Janeiro
6,7,4,2023-01-01,280.0,Otávio Assumpção,Campo Grande


Aqui, temos duas novas vendas, a 6 e a 7, que também refletem as operações realizadas anteriormente.

#### **Questão 05. Iteração em listas e tuplas:**
*Dada uma lista de valores de vendas, itere sobre ela para calcular a média das vendas*

Aqui vamos utilizar nossa prórpia tabela de vendas, a `df_master`. A ideia é pegar todas as vendas realizadas e calcular a média utilizando o laço `for`:

In [81]:
def calcular_media(lista_valores):
    soma = 0
    # Iterando a lista e somando os valores encontrados
    for valor in lista_valores:
        soma += valor
    # Fazendo a média
    media = soma / len(lista_valores)
    return media

# Usando as nossas vendas como exemplo - transforma em lista
valores_vendas = df_master['valor_total'].tolist()

# Chamando a função para calcular a média
media_vendas = calcular_media(valores_vendas)

# Exibindo o resultado
print(f"A média das vendas é: R$ {media_vendas:.2f}")

A média das vendas é: R$ 390.00


#### **Questão 06. Recursão:**
*Implemente uma função recursiva para calcular o fatorial de um número*

Aqui, o exemplo utilizado foi o 5, mas a variável `numero` consegue inputar qualquer outro valor (desde que seja float)

In [85]:
def fatorial(n):
    # Caso base: o fatorial de 0 ou 1 é 1
    if n == 0 or n == 1:
        return 1
    else:
        # Chamada recursiva: n * fatorial(n-1)
        return n * fatorial(n - 1)

# Exemplo de uso
numero = float(input('Numero: '))
resultado = fatorial(numero)
print(f"O fatorial de {numero} é {resultado}")

O fatorial de 5.0 é 120.0
